In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

np.random.seed(1967)

In [2]:
def get_matrix(params):
    """
    Args:
        - params (array): The four parameters of the model.
    Returns:
        - (matrix): The associated matrix to these parameters.
    """

    alpha, beta, gamma, phi = params

    # Put your code here #
    mat = qml.matrix(qml.prod(qml.RZ(gamma, wires=0), qml.RX(beta,wires=0), qml.RZ(alpha, wires=0)))
    mat = np.exp(phi * 1j)*mat
    
    return np.real(mat)

def error(U, params):
    """
    This function determines the similarity between your generated matrix and the target unitary.

    Args:
        - U (matrix): Goal matrix that we want to approach.
        - params (array): The four parameters of the model.

    Returns:
        - (float): Error associated with the quality of the solution.
    """

    matrix = get_matrix(params)
    U = np.array(U)

    # Put your code here #
    sm = 0
    for i in range(2):
        for j in range(2):
            sm += np.absolute(U[i,j] - matrix[i,j])
    
    return sm


In [3]:
def train_parameters(U):

    epochs = 1000
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    params = np.random.rand(4) * np.pi

    for epoch in range(epochs):
        params -= lr * grad(U, params)
        
    return params

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    matrix = json.loads(test_case_input)
    params = [float(p) for p in train_parameters(matrix)]
    return json.dumps(params)

def check(solution_output: str, expected_output: str) -> None:
    matrix1 = get_matrix(json.loads(solution_output))
    matrix2 = json.loads(expected_output)
    assert not np.allclose(get_matrix(np.random.rand(4)), get_matrix(np.random.rand(4)))
    assert np.allclose(matrix1, matrix2, atol=0.2)


test_cases = [['[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]', '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'], ['[[ 1,  0], [ 0, -1]]', '[[ 1,  0], [ 0, -1]]']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'...
Correct!
Running test case 1 with input '[[ 1,  0], [ 0, -1]]'...
Correct!


In [32]:
params = np.random.rand(4) * np.pi
alpha, beta, gamma, phi = params

In [33]:
mat = qml.matrix(qml.prod(qml.RZ(gamma, wires=0), qml.RX(beta,wires=0), qml.RZ(alpha, wires=0)))
mat = np.exp(phi * 1j)*mat

In [5]:
U = np.array([[ 1,  0], [ 0, -1]])

In [8]:
sm = 0
for i in range(2):
    for j in range(2):
        sm += np.absolute(U[i,j] - mat[i,j])


In [46]:
p = train_parameters(U)

[0.38550443 2.31533788 2.77096324 0.93303033]
[0.38749331 2.30575815 2.76892638 0.95148839]
[0.38943699 2.29612859 2.7669337  0.97002186]
[0.39133396 2.28645407 2.76498672 0.98862083]
[0.39318276 2.27673957 2.7630869  1.00727525]
[0.39498201 2.26699016 2.76123566 1.02597491]
[0.39673037 2.25721099 2.75943433 1.04470946]
[0.39842657 2.24740727 2.7576842  1.06346848]
[0.40006943 2.23758428 2.75598648 1.08224146]
[0.40165783 2.22774732 2.7543423  1.10101785]
[0.40319073 2.21790172 2.75275273 1.11978709]
[0.40466716 2.20805284 2.75121874 1.13853866]
[0.40608626 2.198206   2.74974124 1.15726205]
[0.40744723 2.18836652 2.74832104 1.17594686]
[0.39986397 2.17851441 2.75584427 1.17681196]
[0.40113608 2.16868833 2.75451179 1.19548868]
[0.39351136 2.15879371 2.76207538 1.19642736]
[0.38587993 2.1489323  2.76964535 1.19744755]
[0.37824222 2.13910473 2.77722129 1.19854843]
[0.3793685  2.12930711 2.77603292 1.21726877]
[0.37169439 2.11944682 2.78364423 1.21842173]
[0.36401532 2.10962289 2.79126019 

[0.00326963 0.97702619 3.1425234  1.57665617]
[0.00325112 0.9722814  3.14250482 1.56716658]
[0.00323273 0.96757718 3.1424864  1.57657504]
[0.0032145  0.96287474 3.1424681  1.56717016]
[0.0031964  0.95821211 3.14244996 1.57649545]
[0.00317844 0.95355176 3.14243194 1.56717473]
[0.00316061 0.94893045 3.14241407 1.57641736]
[0.00314293 0.9443119  3.14239633 1.56718026]
[3.12537683e-03 9.39731663e-01 3.14237874e+00 1.57634075e+00]
[3.10796859e-03 9.35154639e-01 3.14236127e+00 1.56718669e+00]
[3.09068531e-03 9.30615206e-01 3.14234395e+00 1.57626557e+00]
[3.07354669e-03 9.26079426e-01 3.14232676e+00 1.56719400e+00]
[3.05653213e-03 9.21580536e-01 3.14230970e+00 1.57619179e+00]
[3.03966044e-03 9.17085720e-01 3.14229278e+00 1.56720215e+00]
[3.02291187e-03 9.12627108e-01 3.14227599e+00 1.57611938e+00]
[3.00630437e-03 9.08172976e-01 3.14225933e+00 1.56721111e+00]
[2.98981905e-03 9.03754377e-01 3.14224281e+00 1.57604831e+00]
[2.97347299e-03 8.99340646e-01 3.14222641e+00 1.56722084e+00]
[2.95724812e

[1.75137993e-03 4.95923354e-01 3.14100108e+00 1.56839560e+00]
[1.74576518e-03 4.93445803e-01 3.14099545e+00 1.57335069e+00]
[1.74020360e-03 4.90978767e-01 3.14098987e+00 1.56841663e+00]
[1.73469378e-03 4.88525380e-01 3.14098435e+00 1.57332340e+00]
[1.72923618e-03 4.86082458e-01 3.14097888e+00 1.56843756e+00]
[1.72382944e-03 4.83653015e-01 3.14097346e+00 1.57329644e+00]
[1.71847401e-03 4.81233986e-01 3.14096809e+00 1.56845838e+00]
[1.71316857e-03 4.78828268e-01 3.14096277e+00 1.57326982e+00]
[1.70791352e-03 4.76432911e-01 3.14095750e+00 1.56847910e+00]
[1.70270760e-03 4.74050701e-01 3.14095228e+00 1.57324352e+00]
[1.69755117e-03 4.71678799e-01 3.14094711e+00 1.56849972e+00]
[1.69244302e-03 4.69319883e-01 3.14094199e+00 1.57321755e+00]
[1.68738347e-03 4.66971219e-01 3.14093692e+00 1.56852022e+00]
[1.68237137e-03 4.64635382e-01 3.14093190e+00 1.57319189e+00]
[1.67740700e-03 4.62309741e-01 3.14092692e+00 1.56854061e+00]
[1.67248925e-03 4.59996772e-01 3.14092199e+00 1.57316655e+00]
[1.66761

[1.26869930e-04 2.08494329e-01 3.14167698e+00 1.57186455e+00]
[1.25819689e-04 2.07443121e-01 3.14167593e+00 1.56976213e+00]
[1.24779835e-04 2.06397479e-01 3.14167489e+00 1.57185342e+00]
[1.23750269e-04 2.05356808e-01 3.14167386e+00 1.56977208e+00]
[1.22730888e-04 2.04321639e-01 3.14167284e+00 1.57184241e+00]
[1.21721594e-04 2.03291401e-01 3.14167183e+00 1.56978194e+00]
[1.20722287e-04 2.02266603e-01 3.14167084e+00 1.57183154e+00]
[1.19732871e-04 2.01246694e-01 3.14166985e+00 1.56979172e+00]
[1.18753247e-04 2.00232163e-01 3.14166887e+00 1.57182078e+00]
[1.17783321e-04 1.99222481e-01 3.14166790e+00 1.56980142e+00]
[1.16822997e-04 1.98218116e-01 3.14166694e+00 1.57181015e+00]
[1.15872181e-04 1.97218559e-01 3.14166599e+00 1.56981103e+00]
[1.14930779e-04 1.96224259e-01 3.14166504e+00 1.57179963e+00]
[1.13998700e-04 1.95234727e-01 3.14166411e+00 1.56982057e+00]
[1.13075852e-04 1.94250392e-01 3.14166319e+00 1.57178924e+00]
[1.12162145e-04 1.93270785e-01 3.14166227e+00 1.56983002e+00]
[1.11257

[4.77630202e-05 1.03737768e-01 3.14159787e+00 1.57132224e+00]
[4.74984704e-05 1.03214059e-01 3.14159761e+00 1.57027483e+00]
[4.72365590e-05 1.02693010e-01 3.14159735e+00 1.57131693e+00]
[4.69772598e-05 1.02174572e-01 3.14159709e+00 1.57028005e+00]
[4.67205467e-05 1.01658765e-01 3.14159683e+00 1.57131166e+00]
[4.64663940e-05 1.01145543e-01 3.14159658e+00 1.57028522e+00]
[4.62147764e-05 1.00634927e-01 3.14159632e+00 1.57130645e+00]
[4.59656684e-05 1.00126870e-01 3.14159608e+00 1.57029034e+00]
[4.57190453e-05 9.96213908e-02 3.14159583e+00 1.57130130e+00]
[4.54748822e-05 9.91184465e-02 3.14159558e+00 1.57029541e+00]
[4.52331547e-05 9.86180541e-02 3.14159534e+00 1.57129619e+00]
[4.49938385e-05 9.81201711e-02 3.14159510e+00 1.57030043e+00]
[4.47569097e-05 9.76248140e-02 3.14159487e+00 1.57129114e+00]
[4.45223445e-05 9.71319416e-02 3.14159463e+00 1.57030540e+00]
[4.42901193e-05 9.66415693e-02 3.14159440e+00 1.57128614e+00]
[4.40602109e-05 9.61536571e-02 3.14159417e+00 1.57031032e+00]
[4.38325

[2.72042870e-05 4.87934579e-02 3.14157731e+00 1.57054933e+00]
[2.71449858e-05 4.85470456e-02 3.14157725e+00 1.57104216e+00]
[2.70862774e-05 4.83018766e-02 3.14157720e+00 1.57055182e+00]
[2.70281556e-05 4.80579464e-02 3.14157714e+00 1.57103968e+00]
[2.69706148e-05 4.78152469e-02 3.14157708e+00 1.57055428e+00]
[2.69136490e-05 4.75737737e-02 3.14157702e+00 1.57103723e+00]
[2.68572525e-05 4.73335190e-02 3.14157697e+00 1.57055672e+00]
[2.68014197e-05 4.70944781e-02 3.14157691e+00 1.57103480e+00]
[2.67461450e-05 4.68566433e-02 3.14157686e+00 1.57055913e+00]
[2.66914226e-05 4.66200103e-02 3.14157680e+00 1.57103240e+00]
[2.66372472e-05 4.63845713e-02 3.14157675e+00 1.57056152e+00]
[2.65836132e-05 4.61503218e-02 3.14157669e+00 1.57103002e+00]
[2.65305153e-05 4.59172543e-02 3.14157664e+00 1.57056388e+00]
[2.64779481e-05 4.56853644e-02 3.14157659e+00 1.57102766e+00]
[2.64259063e-05 4.54546447e-02 3.14157653e+00 1.57056622e+00]
[2.63743846e-05 4.52250906e-02 3.14157648e+00 1.57102533e+00]
[2.63233

[2.27547440e-05 2.42612747e-02 3.14157286e+00 1.57067349e+00]
[2.27399217e-05 2.41387430e-02 3.14157285e+00 1.57091855e+00]
[2.27252476e-05 2.40168300e-02 3.14157283e+00 1.57067473e+00]
[2.27107203e-05 2.38955329e-02 3.14157282e+00 1.57091732e+00]
[2.26963382e-05 2.37748482e-02 3.14157281e+00 1.57067595e+00]
[2.26820999e-05 2.36547731e-02 3.14157279e+00 1.57091610e+00]
[2.26680039e-05 2.35353043e-02 3.14157278e+00 1.57067716e+00]
[2.26540489e-05 2.34164389e-02 3.14157276e+00 1.57091489e+00]
[2.26402335e-05 2.32981738e-02 3.14157275e+00 1.57067836e+00]
[2.26265562e-05 2.31805061e-02 3.14157274e+00 1.57091370e+00]
[2.26130156e-05 2.30634325e-02 3.14157272e+00 1.57067955e+00]
[2.25996104e-05 2.29469502e-02 3.14157271e+00 1.57091252e+00]
[2.25863392e-05 2.28310561e-02 3.14157270e+00 1.57068073e+00]
[2.25732008e-05 2.27157474e-02 3.14157268e+00 1.57091135e+00]
[2.25601937e-05 2.26010210e-02 3.14157267e+00 1.57068189e+00]
[2.25473166e-05 2.24868741e-02 3.14157266e+00 1.57091019e+00]
[2.25345

In [42]:
np.real(mat)

tensor([[0.69761082, 0.38615656],
        [0.05524615, 0.56100723]], requires_grad=True)

In [30]:
params

tensor([2.58889016, 0.53545863, 2.11384369, 1.3925301 ], requires_grad=True)